<a href="https://colab.research.google.com/github/BamByeol/learning-ML/blob/HW220122/HW0122_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# <과제1 - 모델 평가 & 분석> => Multi Linear regression
# 주제 : 당뇨병 진행도 예측

In [ ]:
#필요한 패키지들
import torch
import torch.nn as nn

from torch import nn # pytorch 네트워크
import torch.nn.functional as F

#그래프를 그리기 위한 패키지
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#학습데이터를 위한 패키지
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

#pytorch와 비교해볼 scikit-learn 패키지
from sklearn import linear_model
from sklearn.model_selection import train_test_split

from torchvision import models

In [ ]:
#자료출처 : scikit-learn 패키지 제공
#442명의 당뇨병 환자를 대상으로 한 검사 결과 / 10종류의 독립변수 / 종속변수 : 1년뒤 측정한 당뇨병의 진행률
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()
df = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
df["target"] = diabetes.target
df.tail()

#데이터 시각화
#sns.pairplot(df[["target", "bmi", "bp", "s1","age","sex"]]) #산점도그래프를 봤는데 사실 나는 어떤 독립변수 간의 상관관계를 찾기가 어려워보인다..블로그에서는 BMI와 평균혈압이 서로 양의 상관관계가 있고 종속변수인 당뇨병진행도와 양의 상관관계가 있다고 했으니 두 독립변수만 사용하겠다.
#plt.show()

#데이터 전처리 - 칼럼
df = df.drop(columns=["age","sex","s1","s2","s3","s4","s5","s6"]) #필요없는 칼럼 삭제
df.columns

#result
#Index(['bmi', 'bp', 'target'], dtype='object')

#데이터 전처리 - null
print(df.isnull().sum())

#result
#bmi 0
#bp 0
#target 0
#dtype: int64

#데이터 전처리 - 데이터 세트 분리
train,test = train_test_split(df, test_size = 0.2)

x_train_1 = torch.Tensor([[x] for x in list(train.bmi)])
x_train_2 = torch.Tensor([[x] for x in list(train.bp)])
x_train = torch.cat([x_train_1,x_train_2],dim = 1)

y_train = torch.Tensor([[x] for x in list(train.target)])

train_data = TensorDataset(x_train, y_train)

batch_size = len(x_train)
train_dl = DataLoader(train_data, batch_size, shuffle=True)

In [ ]:
#다중선형회귀 모델 생성
model = nn.Linear(2,1)
print(list(model.parameters()))

cost = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) #SGD 경사하강법의 일종

def multimodel(epochs, model, cost_fn, optimizer, train_dl):
    for epoch in range(epochs):
        for xb, yb in train_dl:
            pred = model(xb)
            cost = F.mse_loss(pred, yb)

            optimizer.zero_grad()
            cost.backward()
            optimizer.step()

        print('Epoch [{}/{}], Cost: {:.4f}'.format(epoch+1, epochs, cost.item()))

epochs=1000
multimodel(epochs, model, cost, optimizer, train_dl)

In [ ]:
#데이터시각화
#독립변수가 2개라서 평면 그래프가 아니라 3차원그래프를 그려야 할 것 같음...qwq

#model summary
#model = models.vgg16()
#print(model)


In [ ]:
# w와 b의 값 구하는 코드
regr = linear_model.LinearRegression()
x1 = df.bmi.values.reshape(-1,1)
x2 = df.bp.values.reshape(-1,1)
X = x1 + x2

y = df.target.values.reshape(-1,1)

regr.fit(X, y)
print(regr.coef_[0])
print(regr.intercept_)

# 임의의 입력 [22, 90]를 선언
new_var =  torch.FloatTensor([[22, 90]]) 
# 입력한 값 [22,90]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) 
print("훈련 후 입력이 22,90일 때의 예측값 :", pred_y)

In [ ]:
# 학습을 바탕으로 테스트데이터를 사용하여 w와 b의 값을 찾기
print(list(model.weight))
print(model.bias)

sns.pairplot(df[["target", "bmi", "bp"]])
plt.show()

# 임의의 입력 [22, 90]를 선언
new_var =  torch.FloatTensor([[22, 90]]) 
# 입력한 값 [22,90]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) 
print("훈련 후 입력이 22,90일 때의 예측값 :", pred_y)

#result
#466.4252
#실제 데이터 값은 138인데 독립변수끼리 상관성이 있기 때문일까 학습이 이상하게 된 것 같음
#실제 데이터 값을 볼 수 있는 txt 자료 ( https://www4.stat.ncsu.edu/~boos/var.select/diabetes.tab.txt )

#error 0129
#print(float(model.weight)) -> only one element tensors can be converted to Python scalars

#두 독립변수의 데이터를 합쳤는데 왜 저따구인지 모르겠다. 같은 자료를 가지고 한 다른 블로그 ( https://cocoon1787.tistory.com/175 )를 보면
#저사람도 loss 값을 약 3000까지 줄이긴 하셨는데 